In [1]:
import os
import re
import numpy
import pandas as pd
# import PyPDF2
import requests
from bs4 import BeautifulSoup
from googlesearch import search

In [2]:
base_url = "https://ieeexplore.ieee.org/search/searchresult.jsp"
file = "Quantum_Machine_Learning_for_6G_Communication_Networks:_State-of-the-Art_and_Vision_for_the_Future" 
path = os.getcwd() + "/references/" + file +".txt"
csv_path = os.getcwd() + "/data/" + file +".csv"

In [3]:
csv_data = pd.read_csv(csv_path)
csv_data.head()

,title,author,date,INSPEC,DOI,citations,publisher,url
0,Quantum Machine Learning for 6G Communication ...,"Syed Junaid Nawaz, Shree Krishna Sharma, Shurj...",04 April 2019,18609246,10.1109/ACCESS.2019.2909490,170,IEEE,https://ieeexplore.ieee.org/document/8681450


In [24]:
ref =  open(path,"r", encoding='utf-8') 
while True:
    line = ref.readline()
    if not line: break
    print(line) 
ref.close()

[1] R. S. Michalski, J. G. Carbonell, and T. M. Mitchell, Machine Learning:

An Artificial Intelligence Approach. Berlin, Germany: Springer, 2013.

[2] A. Gupta and E. R. K. Jha, ‘‘A survey of 5G network: Architecture and

emerging technologies,’’ IEEE Access, vol. 3, pp. 1206–1232, Jul. 2015.

[3] C. Bockelmann et al., ‘‘Towards massive connectivity support for scal-

able mMTC communications in 5G networks,’’ IEEE Access, vol. 6,

pp. 28969–28992, May 2018.

[4] S. K. Sharma and X. Wang. (Aug. 2018). ‘‘Towards massive machine

type communications in ultra-dense cellular IoT networks: Current

issues and machine learning-assisted solutions.’’ [Online]. Available:

https://arxiv.org/abs/1808.02924

[5] M. Chiani, E. Paolini, and F. Callegati, ‘‘Open issues and beyond 5G,’’

in 5G Italy White eBook: From Research to Market. Rome, Italy, 2018.

[Online]. Available: https://www.5gitaly.eu/en/white-ebook-2/

[6] K. David and H. Berndt, ‘‘6G vision and requirements: Is there any need

for b

NameError: name 'f' is not defined

In [27]:
def check_index(line,index):
    if(line[0]=="[" and line[1:1+len(str(index))] == str(index)):
        return True
    else:
        return False

def get_paper_html(search_url):
    response = requests.get(search_url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    else:
        return -1

def get_citations(html):
    html = str(html)
    data = 0
    temp = html.split('"citationCount":"')
    if(len(temp)>1):
        data = int(temp[1].split('"')[0])
    return data

def get_INSPEC(html):
    html = str(html)
    data = 0
    temp = html.split('"accessionNumber":"')
    if(len(temp)>1):
        data = int(temp[1].split('"')[0])
    return data

def get_DOI(html):
    html = str(html)
    data = ""
    temp = html.split('"doi":"')
    if(len(temp)>1):
        data = temp[1].split('"')[0]
    return data

def get_date(html):
    html = str(html)
    data = ""
    temp = html.split('"dateOfInsertion":"')
    if(len(temp)>1):
        data = temp[1].split('"')[0]
    return data
    
def handle_data(csv_data,current,index):
    temp = ["","","",0,"",0,"",""]
    current =current.replace("\n","")
#     print("index ",index,"\n",current)
#     parse = re.split('“|”',current)
    parse = re.split('‘‘|’’',current)
    if(len(parse)<2):
        return
    temp[0] = parse[1][:-1]
    temp[1] = parse[0]
    is_url = False
    if("IEEE" in parse[2]):
        temp[6] = "IEEE"
        for search_url in search(temp[0], tld="co.in", num=5, stop=5, pause=1):
            if('ieee' in search_url):
                temp[7] = search_url
                is_url = True
                break;
    if(is_url):
        
        html = get_paper_html(temp[7])
        temp[5] = get_citations(html)
        temp[3] = get_INSPEC(html)
        temp[4] = get_DOI(html)
        temp[2] = get_date(html)
    print("idx ",index," ",temp)
    csv_data.loc[index] = temp
    

def handle_reference(csv_data,file):
    index = 1
    
    current = ""
    while True:
        
        line = file.readline()
        if not line: break
        if(check_index(line, index+1)):
#             print("here")
            handle_data(csv_data,current[3+len(str(index)):],index)
            current = line
            index += 1
        else:
            current += line
            
            
           

In [28]:
ref =  open(path,"r", encoding='utf-8') 

handle_reference(csv_data,ref)
ref.close()

idx  2   ['A survey of 5G network: Architecture andemerging technologies', 'A. Gupta and E. R. K. Jha, ', '07 August 2015', 15351996, '10.1109/ACCESS.2015.2461602', 1308, 'IEEE', 'https://ieeexplore.ieee.org/document/7169508']
idx  3   ['Towards massive connectivity support for scal-able mMTC communications in 5G networks', 'C. Bockelmann et al., ', '18 June 2018', 17829625, '10.1109/ACCESS.2018.2837382', 131, 'IEEE', 'http://ieeexplore.ieee.org/document/8360103/']
idx  4   ['Towards massive machinetype communications in ultra-dense cellular IoT networks: Currentissues and machine learning-assisted solutions', 'S. K. Sharma and X. Wang. (Aug. 2018). ', '', 0, '', 0, '', '']
idx  5   ['Open issues and beyond 5G', 'M. Chiani, E. Paolini, and F. Callegati, ', '', 0, '', 0, '', '']
idx  6   ['6G vision and requirements: Is there any needfor beyond 5G', 'K. David and H. Berndt, ', '30 August 2018', 18058905, '10.1109/MVT.2018.2848498', 247, 'IEEE', 'https://ieeexplore.ieee.org/document/8412

idx  45   ['5G: A tutorial overview of standards, trials, challenges,deployment, and practice', 'M. Shafi et al., ', '02 June 2017', 16915203, '10.1109/JSAC.2017.2692307', 1036, 'IEEE', 'https://ieeexplore.ieee.org/document/7894280']
idx  46   ['What will 5G be', 'J. G. Andrews et al., ', '22 July 2014', 14469237, '10.1109/JSAC.2014.2328098', 5476, 'IEEE', 'https://ieeexplore.ieee.org/document/6824752']
idx  47   ['Millimeter-wave communication with out-of-band information', 'N. Gonzalez-Prelcic, A. Ali, V. Va, and R. W. Heath, Jr., ', '13 December 2017', 17446971, '10.1109/MCOM.2017.1700207', 53, 'IEEE', 'http://ieeexplore.ieee.org/document/8198818']
idx  48   ['Energy and spectral effi-ciency of very large multiuser MIMO systems', 'H. Q. Ngo, E. G. Larsson, and T. L. Marzetta, ', '07 May 2013', 13487669, '10.1109/TCOMM.2013.020413.110848', 2005, 'IEEE', 'https://ieeexplore.ieee.org/document/6457363']
idx  49   ['Optimal cell load and throughput in green small cellnetworks with genera

idx  81   ['Selecting array con-figurations for MIMO systems: An evolutionary computation approach', 'P. D. Karamalis, N. D. Skentos, and A. G. Kanatas, ', '03 January 2005', 8258804, '10.1109/TWC.2004.837447', 30, 'IEEE', 'http://ieeexplore.ieee.org/document/1374907/']
idx  82   ['Power control of cellularradio systems via robust Smith prediction filter', 'B.-K. Lee, H.-W. Chen, and B.-S. Chen, ', '18 October 2004', 8155807, '10.1109/TWC.2004.834705', 39, 'IEEE', 'http://ieeexplore.ieee.org/abstract/document/1343917/']
idx  83   ['Joint receive antenna selection and symboldetection for MIMO systems: A heterogeneous genetic approach', 'H. Y. Lu and W. H. Fang, ', '13 February 2009', 10475211, '10.1109/LCOMM.2009.081036', 7, 'IEEE', 'http://ieeexplore.ieee.org/document/4783770']
idx  84   ['GAbased estimation of sparse MIMO channels with superimposed training', 'B. Mansoor, S. J. Nawaz, M. I. Tiwana, J. Ahmed, and A. Haseeb, ', '', 0, '', 0, '', '']
idx  85   ['Evolutionary algorithms f

idx  124   ['The private classical capacity and quantum capacity of aquantum channel', 'I. Devetak, ', '10 January 2005', 8267891, '10.1109/TIT.2004.839515', 478, 'IEEE', 'https://ieeexplore.ieee.org/iel5/18/30067/01377491.pdf']
idx  125   ['Identifying the informationgain of a quantum measurement', 'M. Berta, J. M. Renes, and M. M. Wilde, ', '11 August 2014', 14515021, '10.1109/ISIT.2014.6874849', 1, 'IEEE', 'http://ieeexplore.ieee.org/document/6874849/']
idx  126   ['Analysis of quantum key distribution basedsatellite communication', 'V. Sharma and S. Banerjee, ', '', 0, '', 0, '', '']
idx  127   ['Quantum key distribution technologies', 'K. Inoue, ', '', 0, '', 0, 'IEEE', '']
idx  128   ['Teleporting an unknown quantum state via dual clas-sical and einstein-podolsky-rosen channels', 'C. H. Bennett, G. Brassard, C. Crépeau, R. Jozsa, A. Peres, andW. K. Wootters, ', '', 0, '', 0, '', '']
idx  129   ['Communication via one- and two-particle operators on Einstein-Podolsky-Rosen states',

idx  167   ['The impact of nodeselfishness on multicasting in delay tolerant networks', 'Y. Li, G. Su, D. O. Wu, D. Jin, L. Su, and L. Zeng, ', '13 June 2011', 12060236, '10.1109/TVT.2011.2149552', 114, 'IEEE', 'http://ieeexplore.ieee.org/document/5762399/']
idx  168   ['Quantum-assistedrouting optimization for self-organizing networks', 'D. Alanis, P. Botsinis, S. X. Ng, and L. Hanzo, ', '23 May 2017', 14429399, '10.1109/ACCESS.2014.2327596', 30, 'IEEE', 'https://ieeexplore.ieee.org/document/6825798']
idx  169   ['Joint channel estimation anddata detection in MIMO-OFDM systems: A sparse Bayesian learningapproach', 'R. Prasad, C. R. Murthy, and B. D. Rao, ', '10 September 2015', 15432970, '10.1109/TSP.2015.2451071', 74, 'IEEE', 'https://ieeexplore.ieee.org/document/7140825']
idx  170   ['Joint channel estima-tion and multiuser detection for SDMA/OFDM based on dual repeatedweighted boosting search', 'J. Zhang, S. Chen, X. Mu, and L. Hanzo, ', '12 September 2011', 12219122, '10.1109/TVT.

idx  215   ['Indoor localization usingvisible light via fusion of multiple classifiers', 'X. Guo, S. Shao, N. Ansari, and A. Khreishah, ', '14 November 2017', 0, '10.1109/JPHOT.2017.2767576', 57, 'IEEE', 'http://ieeexplore.ieee.org/document/8089342']
idx  216   ['Terabit free-space data transmission employing orbitalangular momentum multiplexing', 'J. Wang et al., ', '', 0, '', 0, '', '']
idx  217   ['Adaptive demodulator using machinelearning for orbital angular momentum shift keying', 'J. Li, M. Zhang, and D. Wang, ', '31 July 2017', 17132608, '10.1109/LPT.2017.2726139', 37, 'IEEE', 'http://ieeexplore.ieee.org/document/7976374']
idx  218   ['Analysis of an adaptive orbital angular momentumshift keying decoder based on machine learning under oceanic turbulencechannels', 'X.-Z. Cui et al., ', '', 0, '', 0, '', '']
idx  219   ['Ubiquitous cell-free massive MIMOcommunications', 'G. Interdonato, E. Björnson, H. Q. Ngo, P. Frenger, andE. G. Larsson. (2018). ', '', 0, '', 0, '', '']
idx  22

In [30]:
pd.set_option('display.max_rows', 300)
csv_data.head(300)

,title,author,date,INSPEC,DOI,citations,publisher,url
0,Quantum Machine Learning for 6G Communication ...,"Syed Junaid Nawaz, Shree Krishna Sharma, Shurj...",04 April 2019,18609246,10.1109/ACCESS.2019.2909490,170,IEEE,https://ieeexplore.ieee.org/document/8681450
2,A survey of 5G network: Architecture andemergi...,"A. Gupta and E. R. K. Jha,",07 August 2015,15351996,10.1109/ACCESS.2015.2461602,1308,IEEE,https://ieeexplore.ieee.org/document/7169508
3,Towards massive connectivity support for scal-...,"C. Bockelmann et al.,",18 June 2018,17829625,10.1109/ACCESS.2018.2837382,131,IEEE,http://ieeexplore.ieee.org/document/8360103/
4,Towards massive machinetype communications in ...,S. K. Sharma and X. Wang. (Aug. 2018).,,0,,0,,
5,Open issues and beyond 5G,"M. Chiani, E. Paolini, and F. Callegati,",,0,,0,,
6,6G vision and requirements: Is there any needf...,"K. David and H. Berndt,",30 August 2018,18058905,10.1109/MVT.2018.2848498,247,IEEE,https://ieeexplore.ieee.org/document/8412482
7,A speculative study on 6G,"F. Tariq, M. Khandaker, K.-K. Wong, M. Imran, ...",,0,,0,,
8,6G: The next frontier,"E. C. Strinati, S. Barbarossa, J. L. Gonzalez-...",,0,,0,,
9,"A vision of 6Gwireless systems: Applications, ...","W. Saad, M. Bennis, and M. Chen. (Mar. 2019).",,0,,0,,
11,Thirty years of machine learning: The road to ...,"J. Wang, C. Jiang, H. Zhang, Y. Ren, K.-C. Che...",,0,,0,,


In [31]:
csv_data.to_csv(csv_path,sep=',', na_rep='NaN')